In [1]:
from operator import truediv
from flask import Flask, render_template, url_for, request, redirect, flash
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime
from os.path import exists
from werkzeug.utils import secure_filename
import os
import json

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///whiskey.db'
app.config['UPLOAD_FOLDER'] = './static/images/'
app.secret_key = "super secret key"
ALLOWED_EXTENSIONS = set(['txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'])
db = SQLAlchemy(app)

class Ratings(db.Model):
    rating_num = db.Column(db.Integer, primary_key=True)
    bottle_id = db.Column(db.Integer, nullable=False)
    rating = db.Column(db.Integer, nullable=False)
    drinker = db.Column(db.String(50))
    date_drank = db.Column(db.DateTime, default=datetime.now)
    blind = db.Column(db.Boolean, default=False)
    def __repr__(self):
        return '<rating %r>' % self.rating


class Collection(db.Model):
    bottle_id = db.Column(db.Integer, primary_key=True)
    bottle_name = db.Column(db.String(200))
    whiskey_type = db.Column(db.String(50), default=0)
    proof = db.Column(db.Float, default=0)
    avg_rating = db.Column(db.Float, default=0)
    num_ratings = db.Column(db.Integer, default=0)

    def __repr__(self):
        return '<bottle %r>' % self.bottle
    
class Users(db.Model):
    user_id = db.Column(db.Integer, primary_key=True)
    user_name = db.Column(db.String(50))
    num_ratings = db.Column(db.Integer, default=0)
    num_blinds = db.Column(db.Integer, default=0)
    avg_rating = db.Column(db.Float, default=0)
    avg_blind = db.Column(db.Float, default=0)
    num_bottles = db.Column(db.Integer, default=0)
    
class Owners(db.Model):
    owner_id = db.Column(db.Integer, primary_key=True)
    bottle_id = db.Column(db.Integer, nullable=False)
    user_id = db.Column(db.Integer, nullable=False)
    owns = db.Column(db.Boolean, default=True)
    

/home/mjhampo/Documents/repos/whiskey_db/venv/lib/python3.10/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [6]:
#db.session.rollback()
#db.create_all()
u = db.session.query(Users).filter(Users.user_name=='joe').all()
print(u[0])
db.session.delete(u[0])
db.session.commit()

<Users 15>


In [ ]:

users = db.session.query(Ratings.drinker).distinct().all()
print(users)

user = users[0]

uid = 0
ratings = db.session.query(Ratings.rating, Ratings.drinker, Ratings.blind).filter(Ratings.drinker==user[0]).all()
bratings = db.session.query(Ratings.rating, Ratings.drinker, Ratings.blind).filter(Ratings.drinker==user[0], Ratings.blind==True).all()
rats = [rate.rating for rate in ratings]
brats = [rate.rating for rate in bratings]

if(len(brats)==0):
    newuser = Users(user_id=uid, user_name=user[0], num_ratings=len(ratings), num_blinds=len(bratings), avg_rating=round((sum(rats)/len(rats)), 2), avg_blind=0)
else:
    newuser = Users(user_id=uid, user_name=user[0], num_ratings=len(ratings), num_blinds=len(bratings), avg_rating=round((sum(rats)/len(rats)), 2), avg_blind=round((sum(brats)/len(brats)), 2))
print(ratings)
print(newuser.avg_rating)
db.session.add(newuser)
db.session.commit()


In [ ]:
user = db.session.query(Users).filter(Users.user_id==11).all()
print(user)

db.session.commit()

In [ ]:

def getDuration(then, now = datetime.now(), interval = "default"):

    # Returns a duration as specified by variable interval
    # Functions, except totalDuration, returns [quotient, remainder]

    duration = now - then # For build-in functions
    duration_in_s = duration.total_seconds() 
    days = int(divmod(duration_in_s, 86400)[0])
    hours = int(divmod(duration_in_s, 3600)[0])
    minutes = int(divmod(duration_in_s, 60)[0])
    
    if(days > 0):
        return str(days) + ' days ago'
    elif(hours > 0):
        return str(hours) + ' hours ago'
    elif(minutes > 0):
        return str(minutes) + ' minutes ago'
    else:
        return str(int(divmod(duration_in_s, 1))) + ' seconds ago'
    

In [ ]:
newRatings = db.session.query(Ratings.rating, Ratings.drinker, Ratings.date_drank, Collection.bottle_name).join(Collection, Ratings.bottle_id==Collection.bottle_id).order_by(Ratings.date_drank.desc()).limit(5).all()
rats = [dict(rat) for rat in newRatings]
for rat in rats:
    rat['time'] = getDuration(rat['date_drank'])
print(rats)
print(getDuration(newRatings[0].date_drank))

In [ ]:
import matplotlib.pyplot as plt

ratings = [ r.rating for r in db.session.query(Ratings.rating).all()]
print(sorted(ratings))
edges = []
for i = 
n, b, p = plt.hist(ratings, rwidth=0.75)
plt.bar_label(p)
plt.savefig('./static/figures/hist.png')


print(b)


In [ ]:
bottles = Collection.query.all()
for bot in bottles:
    all_ratings = Ratings.query.filter_by(bottle_id=bot.bottle_id).all()
    all_ratings_num = [rate.rating for rate in all_ratings]
    
    if(len(all_ratings_num) == 0):
        bot.avg_rating = 0
    else:
        bot.avg_rating = round((sum(all_ratings_num)/len(all_ratings_num)), 1)
    
    bot.num_ratings = len(all_ratings_num)
    
db.session.commit()

In [ ]:
rat = Ratings.query.get_or_404(535)
rat.date_drank = datetime(2023,3, 4, 20, 0, 0, 0)
db.session.commit()

In [ ]:
ratings = db.session.query(Ratings.rating, Ratings.drinker, Ratings.date_drank, Collection.bottle_name, Ratings.rating_num).join(Collection, Ratings.bottle_id==Collection.bottle_id).order_by(Ratings.date_drank.desc()).all()

In [ ]:
from datetime import timedelta
for rate in ratings:
    print(rate)
    #rat = Ratings.query.get_or_404(rate[4])
    #print(rat.date_drank)
    #rat.date_drank = rat.date_drank - timedelta(hours=5)
#db.session.commit()